In [ ]:
from nbdev import *
%nbdev_default_export metrics

Cells will be exported to srthesis.metrics,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Metrics

> Metrics used for evaluating model

In addition to metrics traditionally used in image processing, PSNR and SSIM we'll used perception metrics, which are using activations from a pretrained classification AlexNet model, which is 

In [ ]:
%nbdev_export_internal
import fastai
import torch
import os
from fastai.metrics import psnr
from ssim import ssim
from fastai.vision import normalize_funcs, imagenet_stats

os.sys.path.append('./PerceptualSimilarity')
from PerceptualSimilarity import models as LPIPS_models

In [ ]:
%nbdev_export_internal
norm, denorm = normalize_funcs(*imagenet_stats, do_y = True)

In [ ]:
%nbdev_export_internal
def denormalize(func):
    """Decorator dnormalizing image by pushing it to [0,1] range"""
    def _denormalized(pred, targ):
        pred = denorm(pred).clamp(min=0,max=1)
        targ = denorm(targ).clamp(min=0,max=1)
        return func(pred, targ)
    _denormalized.__name__ = func.__name__
    return _denormalized

## Traditional metrics
Metrics traditionally used for image restoration benchmarking are PSNR and SSIM. They not always show the similarity as perceived by human observer, but are easily measurable and suitable for comparing different algorithms on a task with no geometric deformation.

### PSNR

Peak signal to noise ratio

In [ ]:
%nbdev_export

PSNR = denormalize(psnr)


### SSIM

In [ ]:
%nbdev_export
SSIM = denormalize(ssim)

## Perceptual metrics 

Trained neural nets used.

### LPIPS (Learned Perceptual Image Patch Similarity) metric

Paper [The Unreasonable Effectiveness of Deep Features as a Perceptual Metric](https://arxiv.org/abs/1801.03924). In principle it's similar to our Feature Loss, taking activations in this case from Alex Net and fine tuning according to a study with human participants on Amazon Mechanical Turk. There is adden another fully connected layer trying to predict the prefered image among two with different defformations. The activations from this layer are also used.

This metric was used in recent image restoration competitions such as the recent NTIRE 2020 Challenge on Real-World Image Super-Resolution.



In [ ]:
%nbdev_export

_LPIPS_dist = LPIPS_models.PerceptualLoss(model='net-lin', net='alex')

@denormalize
def LPIPS(pred, targ): 
    result = _LPIPS_dist(pred, targ)
    result = torch.mean(result).squeeze()
    return result

Setting up Perceptual loss...
Loading model from: /home/luslad/thesis/PerceptualSimilarity/models/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 0__template.ipynb.
Converted 10_div2k.ipynb.
Converted 20_metrics.ipynb.
Converted 21_loss.ipynb.
Converted augmentations.ipynb.
Converted sr reference.ipynb.
